In [1]:
# defining the database schema using SQLAlchemy 

In [2]:
!pip install pandas sqlalchemy openpyxl mysqlclient pymysql numpy


In [3]:
# connect to MySql using sqlalchemy 
from sqlalchemy import create_engine

database_type = 'mysql'
host = 'localhost'
port = '3306'
database_name = 'formula_1_project' 
username = 'User'
password = 'Formula1_Content_369'

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database_name}', pool_pre_ping=True)

In [4]:
# import the relevant sqlalchemy modules and functions 


In [5]:
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, Date, Time 
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker 
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import registry
from sqlalchemy import Numeric
from sqlalchemy import MetaData
from datetime import datetime
from sqlalchemy.ext.declarative import as_declarative, declared_attr
from sqlalchemy.exc import IntegrityError
from sqlalchemy import UniqueConstraint
from datetime import timedelta

In [6]:
# define the base class 

In [7]:
Base = declarative_base() 

C:\Users\Irene Mwaniki\AppData\Local\Temp\ipykernel_17344\3664959158.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [8]:
# populating the database 

In [9]:
# create a session that connects MySQL to SQLalchemy
engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database_name}', pool_pre_ping=True)
Base.metadata.bind = engine
DBsession = sessionmaker(bind=engine)
session = DBsession()

In [10]:
# define the circuits table 

class Circuit(Base):
    __tablename__ = 'circuits'
    __table_args__ = {'extend_existing': True} 
    circuit_id: Mapped[int] = mapped_column(primary_key =True)
    circuit_name: Mapped[str] =mapped_column(String(255))
    grands_prix: Mapped[str] =mapped_column(String(255))
    location: Mapped[str] =mapped_column(String(255))
    country: Mapped[str] =mapped_column(String(255))
    turns: Mapped[int] =mapped_column(Integer) 
    length_km : Mapped[float] =mapped_column(Numeric(10,3)) 



# create the circuit table
Base.metadata.create_all(engine, checkfirst = True)


# create a session 
Session = sessionmaker(bind=engine)
session = Session()


  
# load the drivers
circuit_file_path = "D:/Data Science Course/F1 project/f1_circuits.xlsx"
circuit_df = pd.read_excel(circuit_file_path)

circuit_df.columns = circuit_df.columns.str.strip()
circuit_df['Length_km'] =(
    circuit_df['Length_km'] 
    .astype(str) # ensures that all values are string 
    .str.replace('km', '', regex=False) # remove the 'km' in the string
    .str.strip() # removes leading and trailing whitespaces 
    .str.replace('\xa0', '', regex=True) # removes non-breaking spaces 
)

for index, row in circuit_df.iterrows():
    circuits = Circuit(
        circuit_id =row['Circuit_id'],
        circuit_name =row['Circuit_Name'],
        grands_prix =row['Grands_Prix'],
        location =row['Location'],
        country =row['Country'],
        turns =row['Turns'],
        length_km =row['Length_km'] 
    )
    session.add(circuits)
session.commit() 
print(f"Circuits table created successfully.")

session.close()

Circuits table created successfully.


In [11]:
# define driver table 
class Driver(Base):
    __tablename__ ='drivers'
    __table_args__ = {'extend_existing': True} 
    first_name: Mapped[str] = mapped_column(String(100))
    last_name: Mapped[str] = mapped_column(String(100))
    driver_name: Mapped[str] = mapped_column(String(255), primary_key = True)
    nationality: Mapped[str] = mapped_column(String(255)) 
    

# create all the tables above 
Base.metadata.create_all(engine, checkfirst = True)


# create a session 
Session = sessionmaker(bind=engine)
session = Session()
    
# load drivers from the excel files 
drivers_file_path = "D:/Data Science Course/F1 project/F1_drivers.xlsx"
drivers_df = pd.read_excel(drivers_file_path)

# data cleaning e.g. columns for missing information and remove white spaces. 
drivers_df.columns = drivers_df.columns.str.strip()

for index, row in drivers_df.iterrows():
    drivers = Driver(
    first_name=row['First_name'],
    last_name=row['Last_name'],
    driver_name=row['Driver_name'],
    nationality=row['nationality']
    )
    session.add(drivers)
session.commit()
print(f"Drivers table created successfully.")

session.close()

    

Drivers table created successfully.


In [12]:
# define winners tables
class Winner(Base):
    __tablename__ = 'winners'
    __table_args__ = {'extend_existing': True} 
    winner: Mapped[str] = mapped_column(String(255)) 
    grand_prix: Mapped[str] = mapped_column(String(255))
    date: Mapped[date] = mapped_column(Date, primary_key = True)
    car: Mapped[str] = mapped_column(String(255))
    laps: Mapped[int] = mapped_column(Integer)
    time: Mapped[time] = mapped_column(Time)


# create all the tables above 
Base.metadata.create_all(engine, checkfirst = True)


# create a session 
Session = sessionmaker(bind=engine)
session = Session()
    
winner_file_path = "D:/Data Science Course/F1 project/f1_data_by_years.xlsx"
winner_df = pd.read_excel(winner_file_path)

winner_df.columns = winner_df.columns.str.strip()

# convert the date column from excel file to the correct date format (python format) 
winner_df['Date'] = pd.to_datetime(winner_df['Date'], format='%d %b %Y', errors ='coerce')

for index, row in winner_df.iterrows():
    winners = Winner(
        grand_prix=row['Grand Prix'], 
        date=row['Date'],
        winner=row['Winner'],
        car=row['Car'],
        laps=row['Laps'],
        time= row['Time']
    )
    session.add(winners)
session.commit()
print("Winners table created successfully." )


session.close()

Winners table created successfully.


In [13]:

# define qualifying table
# define a function that creates a model for each year (2010, 2011 etc) 
def create_qualifying_model(year):
    class QualifyingResult(Base): 
        __tablename__ = f'qualifying_results {year}'
        __table_args__ =  __table_args__ = (UniqueConstraint('position', 'country', 'year', name='_position_country_year_uc'),)
        __table_args__ = {'extend_existing': True}
        id: Mapped[int] = mapped_column(Integer, autoincrement = True, primary_key = True)
        position: Mapped[str] = mapped_column(String(100))
        driver_number: Mapped[int] = mapped_column(Integer)
        driver: Mapped[str] = mapped_column(String(255))
        car: Mapped[str] = mapped_column(String(255))
        Q1: Mapped[str] = mapped_column(String(100))
        Q2: Mapped[str] = mapped_column(String(100))
        Q3: Mapped[str] = mapped_column(String(100))
        laps: Mapped[int] = mapped_column(Integer)
        country: Mapped[str] = mapped_column(String(255)) 
    return QualifyingResult

    
# create all the tables above in the database
Base.metadata.create_all(engine, checkfirst = True)


# create a session 
Session = sessionmaker(bind=engine)
session = Session()

def convert_time_to_timedelta(time_str):
    """Convert time string in MM:SS.s format to timedelta."""
    try:
        dt =pd.to_datetime(time_str, format='%M:%S.%f')
        return timedelta(minuts=dt.minute, seconds=dt.second, milliseconds=dt.microsecond)
    except Exception:
        return None 
        
# list of file paths by year
qualifying_file_paths = {
    2011: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/F1 qualifying 2011.xlsx",
    2012 : "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/f1 qualifying 2012.xlsx",
    2013: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/F1 qualifying 2013.xlsx",
    2014: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/f1 qualifying 2014.xlsx",
    2015: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/F1 qualifying 2015.xlsx",
    2016: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/F1 qualifying 2016.xlsx",
    2017: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/F1 qualifying 2017.xlsx",
    2018: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/F1 qualifying 2018.xlsx",
    2019: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/F1 qualifying 2019.xlsx",
    2020: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/F1 qualifying 2020.xlsx",
    2021: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/f1 qualifying 2021.xlsx",
    2022: "D:/Data Science Course/F1 project/f1 qualifying 2010 - 2022/f1 qualifying 2022.xlsx"
} 

# loop through each year to create the tables and populate with data 
for year, file_path in qualifying_file_paths.items():
    QualifyingResult = create_qualifying_model(year)
    QualifyingResult.__table__.create(bind=engine, checkfirst=True)
    
    # load the data from the all_races excel sheet 
    df = pd.read_excel(file_path, sheet_name="All_races")
    df.columns = df.columns.str.strip() # clean the columns 
    df= df.drop_duplicates(subset=['Position', 'Country', 'Driver Number', 'Driver', 'Driver Abbreviation', 'Car']) 

    # print the first rows to confirm population 
    print(f"Data for year {year}:")
    print(df.head())

    # insert the data for each year 

    for index, row in df.iterrows():
        if pd.isnull(row['Driver']):
            print(f"Skipping row {index} for year {year}: 'Driver'is Null.") 
            continue
        try: 
            qualifying_result = QualifyingResult(
                country=row['Country'],
                position=row['Position'], 
                driver_number=row['Driver Number'],
                driver=row['Driver'],
                car=row['Car'],
                Q1=row['Q1'],
                Q2=row['Q2'],
                Q3=row['Q3'],
                laps=row['Laps']
            )
            session.add(qualifying_result)
        except Exception as e:
            print(f"Error inserting row {index} for year {year}: {e}")
            session.commit()
            print("Qualifying table created successfully." )
session.close()

Data for year 2011:
  Country Position  Driver Number            Driver Driver Abbreviation  \
0  Turkey        1              1  Sebastian Vettel                 VET   
1  Turkey        2              2       Mark Webber                 WEB   
2  Turkey        3              8      Nico Rosberg                 ROS   
3  Turkey        4              3    Lewis Hamilton                 HAM   
4  Turkey        5              5   Fernando Alonso                 ALO   

                       Car        Q1               Q2               Q3  Laps  
0  Red Bull Racing Renault  0.001007         00:01:26  00:01:25.049000    12  
1  Red Bull Racing Renault  0.001008  00:01:26.075000  00:01:25.454000    10  
2                 Mercedes  0.001013  00:01:25.801000  00:01:25.574000    15  
3         McLaren Mercedes  0.001008  00:01:26.066000  00:01:25.595000    13  
4                  Ferrari  0.001011  00:01:26.152000  00:01:25.851000    16  


C:\Users\Irene Mwaniki\AppData\Local\Temp\ipykernel_17344\3354632263.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.QualifyingResult, and will be replaced in the string-lookup table.
  class QualifyingResult(Base):


Data for year 2012:
     Country Position  Driver Number            Driver Driver Abbreviation  \
0  Abu Dhabi        1              4    Lewis Hamilton                 HAM   
1  Abu Dhabi        2              2       Mark Webber                 WEB   
2  Abu Dhabi       RT              1  Sebastian Vettel                 VET   
3  Abu Dhabi        3             18  Pastor Maldonado                 MAL   
4  Abu Dhabi        4              9    Kimi Räikkönen                 RAI   

                       Car               Q1               Q2               Q3  \
0         McLaren Mercedes  00:01:41.497000  00:01:40.901000  00:01:40.630000   
1  Red Bull Racing Renault  00:01:41.933000  00:01:41.277000  00:01:40.978000   
2  Red Bull Racing Renault  00:01:42.160000  00:01:41.511000  00:01:41.073000   
3         Williams Renault  00:01:41.981000  00:01:41.907000  00:01:41.226000   
4            Lotus Renault  00:01:42.222000  00:01:41.532000  00:01:41.260000   

   Laps  
0    15  
1   

C:\Users\Irene Mwaniki\AppData\Local\Temp\ipykernel_17344\3354632263.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.QualifyingResult, and will be replaced in the string-lookup table.
  class QualifyingResult(Base):


Data for year 2013:
  Country Position  Driver Number            Driver Driver Abbreviation  \
0     USA        1              1  Sebastian Vettel                 VET   
1     USA        2              2       Mark Webber                 WEB   
2     USA        3              8   Romain Grosjean                 GRO   
3     USA        4             11   Nico Hulkenberg                 HUL   
4     USA        5             10    Lewis Hamilton                 HAM   

                       Car               Q1               Q2               Q3  \
0  Red Bull Racing Renault  00:01:38.516000  00:01:37.065000  00:01:36.338000   
1  Red Bull Racing Renault  00:01:38.161000  00:01:37.312000  00:01:36.441000   
2            Lotus Renault  00:01:38.676000  00:01:37.523000  00:01:37.155000   
3           Sauber Ferrari  00:01:38.339000  00:01:37.828000  00:01:37.296000   
4                 Mercedes  00:01:37.959000  00:01:37.854000  00:01:37.345000   

   Laps  
0    12  
1    14  
2    17  
3 

...

Data for year 2020:
                      Country  Position  Driver Number            Driver  \
0  Emirates 70th anniversary          1             77   Valtteri Bottas   
1  Emirates 70th anniversary          2             44    Lewis Hamilton   
2  Emirates 70th anniversary          3             27   Nico Hulkenberg   
3  Emirates 70th anniversary          4             33    Max Verstappen   
4  Emirates 70th anniversary          5              3  Daniel Ricciardo   

  Driver Abbreviation                        Car               Q1  \
0                 BOT                   Mercedes  00:01:26.738000   
1                 HAM                   Mercedes  00:01:26.818000   
2                 HUL  Racing Point BWT Mercedes  00:01:27.279000   
3                 VER      Red Bull Racing Honda  00:01:27.154000   
4                 RIC                    Renault  00:01:27.442000   

                Q2               Q3  Laps  
0  00:01:25.785000  00:01:25.154000    20  
1  00:01:26.266000  

C:\Users\Irene Mwaniki\AppData\Local\Temp\ipykernel_17344\3354632263.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.QualifyingResult, and will be replaced in the string-lookup table.
  class QualifyingResult(Base):


Data for year 2021:
  Country Position  Driver Number           Driver Driver Abbreviation  \
0     USA        1             33   Max Verstappen                 VER   
1     USA        2             44   Lewis Hamilton                 HAM   
2     USA        3             11     Sergio Perez                 PER   
3     USA        4             77  Valtteri Bottas                 BOT   
4     USA        5             16  Charles Leclerc                 LEC   

                     Car               Q1               Q2               Q3  \
0  Red Bull Racing Honda  00:01:34.352000  00:01:33.464000  00:01:32.910000   
1               Mercedes  00:01:34.579000  00:01:33.797000  00:01:33.119000   
2  Red Bull Racing Honda  00:01:34.369000  00:01:34.178000  00:01:33.134000   
3               Mercedes  00:01:34.590000  00:01:33.959000  00:01:33.475000   
4                Ferrari  00:01:34.153000  00:01:33.928000  00:01:33.606000   

   Laps  
0    16  
1    16  
2    19  
3    15  
4    17  


C:\Users\Irene Mwaniki\AppData\Local\Temp\ipykernel_17344\3354632263.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.QualifyingResult, and will be replaced in the string-lookup table.
  class QualifyingResult(Base):


Data for year 2022:
     Country Position  Driver Number           Driver Driver Abbreviation  \
0  Australia        1             16  Charles Leclerc                 LEC   
1  Australia        2              1   Max Verstappen                 VER   
2  Australia        3             11     Sergio Perez                 PER   
3  Australia        4              4     Lando Norris                 NOR   
4  Australia        5             44   Lewis Hamilton                 HAM   

                    Car               Q1               Q2               Q3  \
0               Ferrari  00:01:18.881000  00:01:18.606000  00:01:17.868000   
1  Red Bull Racing RBPT  00:01:18.580000  00:01:18.611000  00:01:18.154000   
2  Red Bull Racing RBPT  00:01:18.834000  00:01:18.340000  00:01:18.240000   
3      McLaren Mercedes  00:01:19.280000  00:01:19.066000  00:01:18.703000   
4              Mercedes  00:01:19.401000  00:01:19.106000  00:01:18.825000   

   Laps  
0    21  
1    21  
2    21  
3    20 

In [14]:
# define races table
# define a function that creates a model for each year (2010, 2011 etc) 
def create_race_model(year):
    class RaceResult(Base): 
        __tablename__ = f'race_results {year}'
        __table_args__ =  __table_args__ = (UniqueConstraint('position', 'driver number', 'driver', name='_position_country_year_uc'),)
        __table_args__ = {'extend_existing': True}
        
        id: Mapped[int] = mapped_column(Integer, autoincrement = True, primary_key = True)
        position: Mapped[str] = mapped_column(String(255))
        driver_number: Mapped[int] = mapped_column(Integer)
        driver: Mapped[str] = mapped_column(String(255)) 
        constructor: Mapped[str] = mapped_column(String(255))
        laps: Mapped[str] = mapped_column(String(100))
        country: Mapped[str] = mapped_column(String(255)) 
        
    return RaceResult

    
# create all the table in the database  
Base.metadata.create_all(engine, checkfirst = True)


# create a session 
Session = sessionmaker(bind=engine)
session = Session()

             
# list of file paths by year
race_file_paths = {
    2011: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 Race Results 2011.xlsx",
    2012: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 race results 2012.xlsx",
    2013: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 race results 2013.xlsx",
    2014: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 Race Results 2014.xlsx",
    2015: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 Race Results 2015.xlsx",
    2016: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 Race Results 2016.xlsx",
    2017: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 Race Results 2017.xlsx",
    2018: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 Race Results 2018.xlsx",
    2019: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 Race Results 2019.xlsx",
    2020: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 Race Results 2020.xlsx",
    2021: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 Race Results 2021.xlsx",
    2022: "D:/Data Science Course/F1 project/F1 race list 2010 - 2024/F1 Race Results 2022.xlsx"
    
} 

# loop through each year to create the tables and populate with data 
for year, file_path in race_file_paths.items():
    RaceResult = create_race_model(year)
    RaceResult.__table__.create(bind=engine, checkfirst=True)
    
    # load the data from the all_races excel sheet 
    df = pd.read_excel(file_path, sheet_name="All_races")
    df.columns = df.columns.str.strip() # clean the columns 

    # ensure that the correct column names in dropt_duplicates are provided 
    df= df.drop_duplicates(subset=['Position', 'Country', 'Constructor', 'Driver', 'Driver Number']) 

    # print the first rows to confirm population 
    print(f"Data for year {year}:")
    print(df.head())

    # insert the data for each year 
try: 
    for index, row in df.iterrows():
        if pd.isnull(row['Driver']):
            print(f"Skipping row {index} for year {year}: 'Driver' is NUll.") 
            continue
            
        try: 
            race_result = RaceResult(
                position=row['Position'],
                driver_number=row['Driver Number'], 
                driver=row['Driver'],
                constructor=row['Constructor'], 
                laps=row['Laps'],
                country=row['Country'] 
            )
            session.add(race_result)
        except Exception as e:
            print(f"Error inserting row {index} for year {year}: {e}")
            session.rollback()
            
    session.commit()
    print("Race table created successfully.")
except Exception as e:
    print(f"Error inserting race data: {e}")
    session.rollback()
finally:
    session.close()

Data for year 2011:
     Country Position  Driver Number            Driver  \
0  Singapore        1              1  Sebastian Vettel   
1  Singapore        2              4     Jenson Button   
2  Singapore        3              2       Mark Webber   
3  Singapore        4              5   Fernando Alonso   
4  Singapore        5              3    Lewis Hamilton   

               Constructor Laps         Time  
0  Red Bull Racing-Renault   61  1:59:06.757  
1         McLaren-Mercedes   61       +1.737  
2  Red Bull Racing-Renault   61      +29.279  
3                  Ferrari   61      +55.449  
4         McLaren-Mercedes   61    +1:07.766  


C:\Users\Irene Mwaniki\AppData\Local\Temp\ipykernel_17344\413141162.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.RaceResult, and will be replaced in the string-lookup table.
  class RaceResult(Base):


Data for year 2012:
  Country Position  Driver Number            Driver       Constructor  Laps  \
0   Japan        1              1  Sebastian Vettel  Red Bull-Renault    53   
1   Japan        2              6      Felipe Massa           Ferrari    53   
2   Japan        3             14   Kamui Kobayashi    Sauber-Ferrari    53   
3   Japan        4              3     Jenson Button  McLaren-Mercedes    53   
4   Japan        5              4    Lewis Hamilton  McLaren-Mercedes    53   

  Time/Retired  
0  1:28:56.242  
1      +20.632  
2      +24.538  
3      +25.098  
4      +46.490  


C:\Users\Irene Mwaniki\AppData\Local\Temp\ipykernel_17344\413141162.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.RaceResult, and will be replaced in the string-lookup table.
  class RaceResult(Base):


Data for year 2013:
  Country Position  Driver Number            Driver           Constructor  \
0  Monaco        1              9      Nico Rosberg              Mercedes   
1  Monaco        2              1  Sebastian Vettel      Red Bull-Renault   
2  Monaco        3              2       Mark Webber      Red Bull-Renault   
3  Monaco        4             10    Lewis Hamilton              Mercedes   
4  Monaco        5             15      Adrian Sutil  Force India-Mercedes   

   Laps Time/Retired  
0    78  2:17:52.506  
1    78      + 3.889  
2    78      + 6.314  
3    78     + 13.895  
4    78     + 21.478  


...

Data for year 2020:
   Country Position  Driver Number           Driver  \
0  Hungary        1             44   Lewis Hamilton   
1  Hungary        2             33   Max Verstappen   
2  Hungary        3             77  Valtteri Bottas   
3  Hungary        4             18     Lance Stroll   
4  Hungary        5             23  Alexander Albon   

                 Constructor  Laps Time/Retired  
0                   Mercedes    70  1:36:12.473  
1      Red Bull Racing-Honda    70       +8.702  
2                   Mercedes    70       +9.452  
3  Racing Point-BWT Mercedes    70      +57.579  
4      Red Bull Racing-Honda    70    +1:18.316  


C:\Users\Irene Mwaniki\AppData\Local\Temp\ipykernel_17344\413141162.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.RaceResult, and will be replaced in the string-lookup table.
  class RaceResult(Base):


Data for year 2021:
        Country Position  Driver Number            Driver  \
0  Saudi Arabia        1             44    Lewis Hamilton   
1  Saudi Arabia        2             33    Max Verstappen   
2  Saudi Arabia        3             77   Valtteri Bottas   
3  Saudi Arabia        4             31      Esteban Ocon   
4  Saudi Arabia        5              3  Daniel Ricciardo   

             Constructor  Laps Time/Retired  
0               Mercedes    50  2:06:15.185  
1  Red Bull Racing-Honda    50     +21.8252  
2               Mercedes    50      +27.531  
3         Alpine-Renault    50      +27.633  
4       McLaren-Mercedes    50      +40.121  


C:\Users\Irene Mwaniki\AppData\Local\Temp\ipykernel_17344\413141162.py:4: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.RaceResult, and will be replaced in the string-lookup table.
  class RaceResult(Base):


Data for year 2022:
  Country Position  Driver Number            Driver           Constructor  \
0  Mexico        1              1    Max Verstappen  Red Bull Racing-RBPT   
1  Mexico        2             44    Lewis Hamilton              Mercedes   
2  Mexico        3             11      Sergio Pérez  Red Bull Racing-RBPT   
3  Mexico        4             63    George Russell              Mercedes   
4  Mexico        5             55  Carlos Sainz Jr.               Ferrari   

   Laps Time/Retired  
0    71  1:38:36.729  
1    71      +15.186  
2    71      +18.097  
3    71      +49.431  
4    71      +58.123  
Race table created successfully.
